---

In [6]:
import cv2
import numpy as np
from PIL import Image, ImageChops, ImageEnhance
import os

# ============================================================
# DETECT REAL IMAGE FORMAT
# ============================================================

def detect_format(path):
    """
    Returns:
        "jpeg", "png", "webp", "bmp", etc.
    """
    try:
        with Image.open(path) as img:
            return img.format.lower()   # "jpeg", "png", "webp", etc.
    except:
        return None


# ============================================================
# JPEG ONLY — ERROR LEVEL ANALYSIS (ELA)
# ============================================================

def ela(path, quality=90):
    original = Image.open(path).convert("RGB")

    # Step 1: Recompress at fixed quality
    temp_path = "temp_ela.jpg"
    original.save(temp_path, "JPEG", quality=quality)
    compressed = Image.open(temp_path)

    # Step 2: Difference map
    ela_img = ImageChops.difference(original, compressed)

    # Step 3: Enhance brightness
    extrema = ela_img.getextrema()
    max_diff = max([e[1] for e in extrema])
    scale = 255.0 / max_diff if max_diff != 0 else 1
    ela_img = ImageEnhance.Brightness(ela_img).enhance(scale)

    ela_img.save("ela_result.png")
    return "ela_result.png"


# ============================================================
# JPEG ONLY — DOUBLE JPEG / BLOCK ARTIFACT ANALYSIS
# ============================================================

def block_variance(path):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    h, w = img.shape
    block = 8

    var_map = np.zeros((h // block, w // block))

    for i in range(0, h - block, block):
        for j in range(0, w - block, block):
            patch = img[i:i+block, j:j+block]
            var_map[i//block, j//block] = np.var(patch)

    # Resize back to original size
    var_map = cv2.resize(var_map, (w, h), interpolation=cv2.INTER_NEAREST)
    var_map = cv2.normalize(var_map, None, 0, 255, cv2.NORM_MINMAX)
    cv2.imwrite("block_variance.png", var_map)
    return "block_variance.png"


# ============================================================
# PNG / JPEG — NOISE RESIDUAL MAP
# ============================================================

def noise_map(path):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    blur = cv2.GaussianBlur(img, (5,5), 0)
    noise = cv2.absdiff(img, blur)
    noise = cv2.normalize(noise, None, 0, 255, cv2.NORM_MINMAX)
    cv2.imwrite("noise_map.png", noise)
    return "noise_map.png"


# ============================================================
# PNG / JPEG — SHARPNESS (EDGE) MAP
# ============================================================

def sharpness_map(path):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    lap = cv2.Laplacian(img, cv2.CV_64F)
    lap = np.absolute(lap)
    lap = cv2.normalize(lap, None, 0, 255, cv2.NORM_MINMAX)
    cv2.imwrite("sharpness_map.png", lap)
    return "sharpness_map.png"


# ============================================================
# MAIN FORENSIC PIPELINE
# ============================================================

def forensic_pipeline(path):

    fmt = detect_format(path)
    print(f"[INFO] Detected format: {fmt}")

    results = {}

    # JPEG-specific methods
    if fmt == "jpeg":
        print("[INFO] Running JPEG forensic methods...")
        results["ela"] = ela(path)
        results["block_variance"] = block_variance(path)

    else:
        print("[INFO] JPEG-specific methods skipped (not jpeg).")

    # Methods for all formats
    print("[INFO] Running universal forensic methods...")
    results["noise_map"] = noise_map(path)
    results["sharpness_map"] = sharpness_map(path)

    print("\n[INFO] Forensic analysis complete. Results saved:")
    for k, v in results.items():
        print(f" - {k}: {v}")
        
    heatmap_path = tampering_heatmap(path, results)
    results["heatmap"] = heatmap_path

    return results

# ============================================================
# COMPUTE FORENSIC SCORES
# ============================================================

def compute_scores(results):
    scores = {}

    # 1. ELA Score (JPEG only)
    if "ela" in results:
        ela_img = cv2.imread(results["ela"], cv2.IMREAD_GRAYSCALE)
        scores["ela_mean"] = float(np.mean(ela_img))
    else:
        scores["ela_mean"] = 0  # PNG → no ELA

    # 2. Noise Score
    noise = cv2.imread(results["noise_map"], cv2.IMREAD_GRAYSCALE)
    scores["noise_std"] = float(np.std(noise))

    # 3. Sharpness Score
    sharp = cv2.imread(results["sharpness_map"], cv2.IMREAD_GRAYSCALE)
    scores["sharpness_mean"] = float(np.mean(sharp))

    return scores


# ============================================================
# SIMPLE DECISION ENGINE (RULE-BASED)
# ============================================================

def decide_fakeness(scores, fmt):
    
    # Thresholds — simple demo values
    ELA_THRESHOLD = 40
    NOISE_THRESHOLD = 25
    SHARPNESS_THRESHOLD = 60

    ela_flag = scores["ela_mean"] > ELA_THRESHOLD if fmt == "jpeg" else False
    noise_flag = scores["noise_std"] > NOISE_THRESHOLD
    sharp_flag = scores["sharpness_mean"] > SHARPNESS_THRESHOLD

    # Fraud score = number of triggered rules
    fraud_score = ela_flag + noise_flag + sharp_flag

    if fraud_score >= 2:
        result = "FAKE"
    else:
        result = "REAL"

    return result, fraud_score

# ============================================================
# TAMPERING LOCALIZATION HEATMAP
# ============================================================

def tampering_heatmap(path, results):
    """
    Combines noise, sharpness, ELA (if available), and block variance (if available)
    into a single anomaly heatmap.
    """
    # Load original
    orig = cv2.imread(path)
    h, w = orig.shape[:2]

    # Load maps (some may not exist)
    noise = cv2.imread(results["noise_map"], cv2.IMREAD_GRAYSCALE)
    sharp = cv2.imread(results["sharpness_map"], cv2.IMREAD_GRAYSCALE)

    # Normalize
    noise_norm = noise.astype(np.float32) / 255.0
    sharp_norm = sharp.astype(np.float32) / 255.0

    anomaly = (noise_norm * 0.5) + (sharp_norm * 0.5)

    # JPEG-only layers
    if "ela" in results:
        ela = cv2.imread(results["ela"], cv2.IMREAD_GRAYSCALE)
        ela_norm = ela.astype(np.float32) / 255.0
        anomaly += ela_norm * 0.7

    if "block_variance" in results:
        bv = cv2.imread(results["block_variance"], cv2.IMREAD_GRAYSCALE)
        bv_norm = bv.astype(np.float32) / 255.0
        anomaly += bv_norm * 0.4

    # Normalize safely
    maxval = anomaly.max()
    if maxval > 0:
        anomaly = anomaly / maxval

    # Smooth
    anomaly = cv2.GaussianBlur(anomaly, (21, 21), 0)

    # Heatmap
    heatmap = (anomaly * 255).astype(np.uint8)
    heatmap_color = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    heatmap_color = cv2.resize(heatmap_color, (w, h))

    overlay = cv2.addWeighted(orig, 0.6, heatmap_color, 0.4, 0)

    cv2.imwrite("heatmap_gray.png", heatmap)
    cv2.imwrite("heatmap_overlay.png", overlay)

    return "heatmap_overlay.png"



# ============================================================
# RUNNER
# ============================================================

image_path = "../../input/fake-2.jpg"
results = forensic_pipeline(image_path)
fmt = detect_format(image_path)

scores = compute_scores(results)
print("\n[INFO] Forensic scores:", scores)

decision, fraud_score = decide_fakeness(scores, fmt)
print(f"\n[RESULT] Decision: {decision} (fraud score = {fraud_score})")

[INFO] Detected format: jpeg
[INFO] Running JPEG forensic methods...
[INFO] Running universal forensic methods...

[INFO] Forensic analysis complete. Results saved:
 - ela: ela_result.png
 - block_variance: block_variance.png
 - noise_map: noise_map.png
 - sharpness_map: sharpness_map.png

[INFO] Forensic scores: {'ela_mean': 2.0112096943048576, 'noise_std': 25.687977778493764, 'sharpness_mean': 7.175243404522613}

[RESULT] Decision: REAL (fraud score = 1)
